In [9]:
import random
import pandas as pd
import torch
import copy
import os
import numpy as np
from sklearn.model_selection import train_test_split
from itertools import product
from sklearn.metrics import r2_score
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.decomposition import PCA

def fix_random(seed):
    torch.manual_seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
fix_random(42)

pca_t = True


In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
#import dataset
df = pd.read_csv('dataset-ml-25m/dataset.csv')

In [12]:
# Dividi il dataset in feature e target
X = df.drop(['rating'], axis=1).to_numpy()
y = df['rating'].to_numpy()

# Dividi il dataset in training, validation e test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
#count the numebr of x_train 
print("Number of train set: ", X_train.shape[0])
print("Numebr of test set: ", X_test.shape[0])
print("Number of validation set: ", X_val.shape[0])


Number of train set:  9946
Numebr of test set:  2764
Number of validation set:  1106


In [13]:
# Applica la riduzione della dimensionalità con PCA
if pca_t == True:
    pca = PCA(n_components=0.95)
    pca.fit(X_train)
    X_train = pca.transform(X_train)
    X_val = pca.transform(X_val)
    X_test = pca.transform(X_test)
else:
    print ("PCA is not applied")

In [14]:
batch_sizes = [256]
n_epochs = [200]
# Dimension of the prediction  layer
n_d = [8, 16, 32]
#Dimension of the attention  layer
n_a = [8, 16, 32]
#Number of successive steps in the network
n_steps = [3, 5, 7]
# Number of independent GLU layer in each GLU block
n_indipendent = [2, 3]

params = list(product(batch_sizes, n_epochs, n_d, n_a, n_steps, n_indipendent))
comb = len(batch_sizes)*len(n_d)*len(n_a) *len(n_steps) *len(n_indipendent)
print("Number of combinations: ", comb)

Number of combinations:  54


In [15]:
def get_model(batch_size, n_epochs, n_d, n_a, n_steps, n_indipendent):
    model = TabNetRegressor(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        n_independent=n_indipendent
    )
    return model

In [16]:
best_mse = float('inf')
best_model = None
best_params = None
iter = 0
for b, n_e, n_d, n_a, n_s, n_i in params:
    iter += 1
    print(f'\nIteration {iter}/{comb}')
    print(f"Configuration batch size: {b} - epochs: {n_e} - n_d: {n_d} - n_a: {n_a} - steps: {n_s} - n_indipendent: {n_i}")
    
    model = get_model(b, n_e, n_d, n_a, n_s, n_i)
    #save results for each iteration with tensorboard
    log = f'bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}'

    if pca_t == True:
        if os.path.exists("./tabpca/"+log):
            print("Model already trained. Skipping...")
            continue
        writer = SummaryWriter(f'tab_pca/bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}')
    else:
        if os.path.exists("./tab_no_pca/"+log):
            print("Model already trained. Skipping...")
            continue
        writer = SummaryWriter(f'tab_no_pca/bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}')

    #fit model
    model.fit(
        X_train=X_train,
        y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_name=['mse'],
        patience=10,
        batch_size=b
    )

    # evaluate model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
   #save hparams for each iteration with tensorboard
    writer.add_hparams(
        {'batch_size': b, 'n_epochs': n_e, 'n_d': n_d, 'n_a': n_a, 'n_steps': n_s, 'n_indipendent': n_i},
        {'hparam/mse': mse, 'hparam/r2': r2}
    )

    
    print('MSE:', mse)
    print('R2 Score:', r2)
    if mse < best_mse:
        best_mse = mse
        best_model = copy.deepcopy(model)
        best_params = (b, n_e, n_d, n_a, n_s, n_i)
        print('Best model updated')
    writer.close()
            


Iteration 1/54
Configuration batch size: 256 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 - n_indipendent: 2


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.4431  | mse_mse: 0.55601 |  0:00:02s
epoch 1  | loss: 0.27711 | mse_mse: 0.30228 |  0:00:04s
epoch 2  | loss: 0.25628 | mse_mse: 0.27185 |  0:00:07s
epoch 3  | loss: 0.24273 | mse_mse: 0.25033 |  0:00:09s
epoch 4  | loss: 0.24211 | mse_mse: 0.25804 |  0:00:12s
epoch 5  | loss: 0.24092 | mse_mse: 0.24931 |  0:00:15s
epoch 6  | loss: 0.22846 | mse_mse: 0.22774 |  0:00:18s
epoch 7  | loss: 0.21858 | mse_mse: 0.20714 |  0:00:21s
epoch 8  | loss: 0.18961 | mse_mse: 0.15768 |  0:00:23s
epoch 9  | loss: 0.14221 | mse_mse: 0.17193 |  0:00:26s
epoch 10 | loss: 0.10433 | mse_mse: 0.08337 |  0:00:29s
epoch 11 | loss: 0.0792  | mse_mse: 0.05475 |  0:00:32s
epoch 12 | loss: 0.06385 | mse_mse: 0.04705 |  0:00:36s
epoch 13 | loss: 0.05083 | mse_mse: 0.04002 |  0:00:38s
epoch 14 | loss: 0.04332 | mse_mse: 0.04124 |  0:00:40s
epoch 15 | loss: 0.04216 | mse_mse: 0.04099 |  0:00:43s
epoch 16 | loss: 0.03664 | mse_mse: 0.02717 |  0:00:45s
epoch 17 | loss: 0.02981 | mse_mse: 0.02608 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.006771515136360431
R2 Score: 0.9694517324604843
Best model updated

Iteration 2/54
Configuration batch size: 256 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 - n_indipendent: 3


...

MSE: 0.008578732406097639
R2 Score: 0.961298851525238

Iteration 54/54
Configuration batch size: 256 - epochs: 200 - n_d: 32 - n_a: 32 - steps: 7 - n_indipendent: 3


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.67803 | mse_mse: 1.3701  |  0:00:05s
epoch 1  | loss: 1.01698 | mse_mse: 0.68618 |  0:00:10s
epoch 2  | loss: 0.44295 | mse_mse: 0.40575 |  0:00:16s
epoch 3  | loss: 0.42806 | mse_mse: 0.513   |  0:00:21s
epoch 4  | loss: 0.27217 | mse_mse: 0.58956 |  0:00:27s
epoch 5  | loss: 0.27568 | mse_mse: 0.26442 |  0:00:32s
epoch 6  | loss: 0.24842 | mse_mse: 0.22992 |  0:00:37s
epoch 7  | loss: 0.25071 | mse_mse: 0.24684 |  0:00:43s
epoch 8  | loss: 0.24663 | mse_mse: 0.255   |  0:00:48s
epoch 9  | loss: 0.24942 | mse_mse: 0.24417 |  0:00:53s
epoch 10 | loss: 0.24433 | mse_mse: 0.22656 |  0:00:59s
epoch 11 | loss: 0.24142 | mse_mse: 0.23637 |  0:01:04s
epoch 12 | loss: 0.24151 | mse_mse: 0.24534 |  0:01:10s
epoch 13 | loss: 0.23701 | mse_mse: 0.21686 |  0:01:15s
epoch 14 | loss: 0.2349  | mse_mse: 0.21894 |  0:01:20s
epoch 15 | loss: 0.22878 | mse_mse: 0.21198 |  0:01:26s
epoch 16 | loss: 0.22307 | mse_mse: 0.21177 |  0:01:31s
epoch 17 | loss: 0.22658 | mse_mse: 0.2177  |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.010589293649987022
R2 Score: 0.9522286269822673


In [17]:
#pint the best model
print(best_model)
print("Best MSE: ", best_mse)
print("R2 score", r2_score(y_test, best_model.predict(X_test)))


TabNetRegressor(n_d=8, n_a=16, n_steps=3, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=552, output_dim=1, device_name='auto', n_shared_decoder=1, n_indep_decoder=1)
Best MSE:  0.006645834471059633
R2 score 0.9700187143708603


In [18]:
#save the best model in a file csv
best_model.save_model('best_model_tabnet_pca_256.csv')

Successfully saved model at best_model_tabnet_pca_256.csv.zip


'best_model_tabnet_pca_256.csv.zip'